In [ ]:
import pandas as pd

# Merge HTS  and PMTCT data
hts_df = pd.read_csv('./clean_data/clean_hts.csv')
pmtct_df = pd.read_csv('./clean_data/merged_pmtct.csv')

# hts_merged = pd.concat([hts_migori, hts_kakamega], ignore_index=True)
hts_df.head()

In [ ]:
hts_df = hts_df.groupby(['facilityuid', 'period']).agg({'dhis2_value': 'sum', 'datim_value': 'sum'})

hts_df

In [ ]:
hts_df = hts_df.reset_index()
hts_df

In [ ]:
hts_df = hts_df.rename(columns={'dhis2_value': 'total_dhis2_value_hts', 'datim_value': 'total_datim_value_hts'})
hts_df

In [ ]:
hts_df['difference_hts'] = hts_df['total_dhis2_value_hts'] - hts_df['total_datim_value_hts']
hts_df

In [ ]:
hts_df['percentage_difference_hts'] = abs(hts_df['difference_hts'] / hts_df['total_datim_value_hts'] * 100)
hts_df

# PMTCT

In [ ]:
# hts_merged = pd.concat([hts_migori, hts_kakamega], ignore_index=True)
pmtct_df.head()

In [ ]:
pmtct_df = pmtct_df.groupby(['facilityuid', 'period']).agg({'dhis2_value': 'sum', 'datim_value': 'sum'})

pmtct_df

In [ ]:
pmtct_df = pmtct_df.reset_index()
pmtct_df

In [ ]:
pmtct_df = pmtct_df.rename(columns={'dhis2_value': 'total_dhis2_value_pmtct', 'datim_value': 'total_datim_value_pmtct'})
pmtct_df

In [ ]:
pmtct_df['difference_pmtct'] = pmtct_df['total_dhis2_value_pmtct'] - pmtct_df['total_datim_value_pmtct']
pmtct_df

In [ ]:
pmtct_df['percentage_difference_pmtct'] = abs(pmtct_df['difference_pmtct'] / pmtct_df['total_datim_value_pmtct'] * 100)
pmtct_df

# Merge PMTCT and HTS

In [ ]:
merged_df = pd.merge(hts_df, pmtct_df, on=["facilityuid","period"])
merged_df

In [ ]:
# calculate mean
cutoff = merged_df["percentage_difference_hts"].median()
print(cutoff)
z = merged_df[merged_df['percentage_difference_hts'] > cutoff]
z

In [ ]:
# define a lambda function with if-else conditions
merged_df['accept'] = merged_df['percentage_difference_hts'].apply(lambda x: 'yes' if x < cutoff else 'no')
merged_df

In [ ]:
len(merged_df[merged_df['accept'] == 'no'])


In [ ]:
rejected = merged_df[merged_df['accept'] == 'no']
rejected

In [ ]:

if (rejected['difference_hts'] < 0).any():
    rejected['new_difference'] = rejected['total_dhis2_value_pmtct'] + rejected['difference_hts']
else:
    rejected['new_difference'] = rejected['total_datim_value_pmtct'] + rejected['difference_hts']

rejected

In [ ]:
rejected['new_percentage_difference']= abs(rejected['new_difference']/ rejected['total_datim_value_hts']* 100)
rejected

In [ ]:
new_cutoff = rejected["new_percentage_difference"].mean()
print(new_cutoff)
b = rejected[rejected['new_percentage_difference'] > new_cutoff]
b

In [ ]:
# define a lambda function with if-else conditions
rejected['new_accept'] = rejected['new_percentage_difference'].apply(lambda x: 'yes' if x < new_cutoff else 'no')
rejected

In [ ]:
rejected['accept'] = rejected['new_accept']
rejected

In [ ]:
merged_df = merged_df[merged_df['accept'] != 'no'].copy()
merged_df

In [ ]:
rejected.drop(['new_difference','new_percentage_difference', 'new_accept'], axis=1, inplace=True)
rejected

In [ ]:
final_merge = pd.concat([merged_df, rejected], axis=0)
final_merge

In [ ]:
len(final_merge[final_merge['accept'] == 'no'])